In [ ]:
# import modules
from utils import FaceRecognition

In [ ]:
# Enter test images file names
group_img = raw_input("Please, enter group image file name: ")
single_img = raw_input("Please, enter single image file name: ")

# or by default
if group_img == "": group_img = "../test/g3.jpg"
if single_img == "": single_img = "../test/s3.jpg"    

In [ ]:
# Read images files
# G - image of group of people
# S - image of a single person
G = cv2.imread(group_img)
S = cv2.imread(single_img)

In [ ]:
# Plot RGB images
# plot group image
fig = plt.figure(figsize=(15,15))
plt.subplot(121)
plt.imshow(cv2.cvtColor(G, cv2.COLOR_BGR2RGB))    
plt.axis("off")

# plot single image
plt.subplot(122)
plt.imshow(cv2.cvtColor(S, cv2.COLOR_BGR2RGB))    
plt.axis("off")

In [ ]:
# Convert original RGB images to grayscale
G_gray = cv2.cvtColor(G, cv2.COLOR_BGR2GRAY)
S_gray = cv2.cvtColor(S, cv2.COLOR_BGR2GRAY)

In [ ]:
# Plot grayscale images
# plot group image
fig = plt.figure(figsize=(15,15))
plt.subplot(121)
plt.imshow(cv2.cvtColor(G_gray, cv2.COLOR_GRAY2RGB))    
plt.axis("off")

# plot single image
plt.subplot(122)
plt.imshow(cv2.cvtColor(S_gray, cv2.COLOR_GRAY2RGB))    
plt.axis("off")

In [ ]:
fr = FaceRecognition()
fr.process(group_img, single_img)

In [ ]:
# plot group image
fig = plt.figure(figsize=(15,15))
plt.subplot(121)
plt.imshow(cv2.cvtColor(G, cv2.COLOR_BGR2RGB))    
plt.axis("off")

# plot single image
plt.subplot(122)
plt.imshow(cv2.cvtColor(S, cv2.COLOR_BGR2RGB))    
plt.axis("off")

In [ ]:
# Mean face size
x_mean = int(math.floor(G_faces[:,3].mean()))

In [ ]:
# Create matrix 
X = np.zeros(shape=(G_faces.shape[0], x_mean**2))
X_test = np.zeros(shape=(S_faces.shape[0], x_mean**2))

<img src="rotation_matrix.png"/>
or
<img src="affine_transformation_matrix.png"/>

In [ ]:
# Align group faces
for i,(x,y,w,h) in enumerate(G_faces):
    # Extract face
    img = G_gray[y:y+h, x:x+w]

    # Align faces
    if apply_align_face:
        img = align_face(G_gray, (x,y,w,h), i, G_faces.shape[0])

    # resize image
    res = cv2.resize(img, (x_mean, x_mean), interpolation=cv2.INTER_CUBIC)

    # LBP
    radius = 3
    n_points = 24
    if apply_lpb: 
        from skimage.feature import local_binary_pattern
        res = local_binary_pattern(res, n_points, radius, method="uniform")

    # Unfold matrix to vector
    res1 = res.reshape(1, res.shape[0]*res.shape[1])

    # Add image vector to the matrix
    X[i,:] = res1

In [ ]:
# Align single face
for i,(x,y,w,h) in enumerate(S_faces):
    # Extract face
    img = S_gray[y:y+h, x:x+w]

    # Align faces
    if apply_align_face:
        img = align_face(S_gray, (x,y,w,h), i, S_faces.shape[0])
    
    # resize image
    res = cv2.resize(img, (x_mean, x_mean), interpolation=cv2.INTER_CUBIC)

    # LBP
    if apply_lpb: 
        from skimage.feature import local_binary_pattern
        res = local_binary_pattern(res, n_points, radius, method="uniform")

    # Unfold matrix to vector
    res1 = res.reshape(1, res.shape[0]*res.shape[1])

    # Add image vector to the matrix
    X_test[i,:] = res1

In [ ]:
# Face indentification
G = cv2.imread(group_img)
S = cv2.imread(single_img)
for i, I in enumerate(X_test):
    D = X - I
    N = np.linalg.norm(D**2, axis=-1)
    idx = np.argmin(N)
    (x,y,w,h) = G_faces[idx]
    (xs,ys,ws,hs) = S_faces[i]
    cv2.rectangle(G,(x,y),(x+w,y+h),(0,0,255),4)
    cv2.rectangle(S,(xs,ys),(xs+ws,ys+hs),(0,0,255),4)

# plot image
fig = plt.figure(figsize=(15,15))
plt.axis("off")
plt.imshow(cv2.cvtColor(G, cv2.COLOR_BGR2RGB))    

In [ ]:
print X[0].shape
print I.shape
print N
print img[0,:10]

In [ ]:
import math
for i, val in enumerate(X):
    A = val
    B = X_test[0]
    nom = (A-B)**2
    den = A+B
    d = math.fsum(nom[den!=0]/den[den!=0])
    print d
#chisquare(I,X[0], axis=None)